In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-indian-recipes-dataset/Cleaned_Indian_Food_Dataset.csv


In [43]:
df=pd.read_csv('/kaggle/input/cleaned-indian-recipes-dataset/Cleaned_Indian_Food_Dataset.csv')
df.head(1)

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10


In [9]:
df.shape
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5938 entries, 0 to 5937
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   TranslatedRecipeName    5938 non-null   object
 1   TranslatedIngredients   5938 non-null   object
 2   TotalTimeInMins         5938 non-null   int64 
 3   Cuisine                 5938 non-null   object
 4   TranslatedInstructions  5938 non-null   object
 5   URL                     5938 non-null   object
 6   Cleaned-Ingredients     5938 non-null   object
 7   image-url               5938 non-null   object
 8   Ingredient-count        5938 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 417.6+ KB
None


In [8]:
df.TranslatedInstructions[1]

'To make tomato puliogere, first cut the tomatoes.\nNow put in a mixer grinder and puree it.\nNow heat oil in a pan.\nAfter the oil is hot, add chana dal, urad dal, cashew and let it cook for 10 to 20 seconds.\nAfter 10 to 20 seconds, add cumin seeds, mustard seeds, green chillies, dry red chillies and curry leaves.\nAfter 30 seconds, add tomato puree to it and mix.\nAdd BC Belle Bhat powder, salt and mix it.\nAllow to cook for 7 to 8 minutes and then turn off the gas.\nTake it out in a bowl, add cooked rice and mix it.\nServe hot.\nServe tomato puliogre with tomato cucumber raita and papad for dinner.'

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
vectorizer = TfidfVectorizer()

ingredient_vectors = vectorizer.fit_transform(df['Cleaned-Ingredients'])

In [19]:
# Input ingredients
input_ingredients = ["chicken", "rice"]
input_vector = vectorizer.transform([" ".join(input_ingredients)])

In [20]:
# Calculate cosine similarity
similarities = cosine_similarity(input_vector, ingredient_vectors)
similarities

array([[0.        , 0.07829381, 0.10711329, ..., 0.        , 0.        ,
        0.183301  ]])

In [21]:
top_n = similarities.argsort()[0][-5:][::-1]
recommendations = df.iloc[top_n]

In [29]:
# Print recommended recipes
recommendations.TranslatedRecipeName

1528                 Chicken In Tomato Onion Gravy Recipe
5259                        Dragon Chicken Sizzler Recipe
3453    Sel Roti Recipe (Nepalese ring-Shaped Sweet br...
3012                  Restaurant Style Chicken 555 Recipe
4717    Malabar Style Kozhi Pidi Recipe - Rice Dumplin...
Name: TranslatedRecipeName, dtype: object

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [3]:
# Load dataset
data = pd.read_csv("/kaggle/input/cleaned-indian-recipes-dataset/Cleaned_Indian_Food_Dataset.csv")

# Ensure ingredients are clean
data['Cleaned-Ingredients'] = data['Cleaned-Ingredients'].str.lower()
data['Cleaned-Ingredients'] = data['Cleaned-Ingredients'].str.replace(',', ' ')

data['TranslatedRecipeName']=data['TranslatedRecipeName'].str.lower()

# Initialize vectorizer for Cleaned-Ingredients
ingredients_vectorizer = TfidfVectorizer()
query_vectorizer = TfidfVectorizer()


# Fit and transform the Cleaned-Ingredients column
ingredient_vectors = ingredients_vectorizer.fit_transform(data['Cleaned-Ingredients'])
query_vectors=query_vectorizer.fit_transform(data['TranslatedRecipeName'])

# Save the vectorizer for later use
with open('ingredient_vectorizer.pkl', 'wb') as f:
    pickle.dump(ingredient_vectors, f)
with open('query_vectorizer.pkl', 'wb') as f:
    pickle.dump(query_vectors, f)

In [8]:
class RecipeFinder:
    def __init__(self, recipe_data, ingredient_vectorizer, query_vectorizer, ingredient_vectors, query_vectors):
        self.recipe_data = recipe_data  # The original recipe dataset
        self.ingredient_vectorizer = ingredient_vectorizer  # TF-IDF vectorizer for ingredients
        self.query_vectorizer = query_vectorizer  # TF-IDF vectorizer for recipe names
        self.ingredient_vectors = ingredient_vectors  # TF-IDF vectors for the ingredients
        self.query_vectors = query_vectors  # TF-IDF vectors for the recipe names

    def find_recipes(self, input_ingredients=None, input_query=None, top_n=5):
        # Initialize cosine similarity for ingredients (if input_ingredients is provided)
        if input_ingredients:
            ingredient_vector = self.ingredient_vectorizer.transform([" ".join(input_ingredients)])
            ingredient_similarities = cosine_similarity(ingredient_vector, self.ingredient_vectors)
        else:
            # If no ingredients are provided, set similarity to zero for all recipes
            ingredient_similarities = [[0] * len(self.recipe_data)] 

        # Initialize cosine similarity for recipe names (if input_query is provided)
        if input_query:
            query_vector = self.query_vectorizer.transform([input_query])
            query_similarities = cosine_similarity(query_vector, self.query_vectors)
        else:
            # If no name query, set similarity to zero for all recipes
            query_similarities = [[0] * len(self.recipe_data)] 

        print(ingredient_similarities,query_similarities)
        # Combine the similarity scores (simple average of both attributes)
        combined_similarities = (ingredient_similarities + query_similarities) / 2

        # Get the top N indices based on the combined similarity
        top_indices = combined_similarities.argsort()[0][-top_n:][::-1]

        # Handle case where fewer recipes exist
        top_indices = top_indices[:min(len(self.recipe_data), top_n)]

        # Return top N recipes
        return self.recipe_data.iloc[top_indices]


In [9]:
# Initialize RecipeFinder with preprocessed data
recipe_finder = RecipeFinder(data, ingredients_vectorizer, query_vectorizer, ingredient_vectors, query_vectors)

# Save the model
with open('recipe_finder_model.pkl', 'wb') as f:
    pickle.dump(recipe_finder, f)


In [10]:
with open('recipe_finder_model.pkl', 'rb') as f:
    recipe_finder = pickle.load(f)  

# Find recipes using the model
input_ingredients = input('Enter ingredients (space-separated) [optional]: ').split() if input('Do you want to enter ingredients? (yes/no): ').lower() == 'yes' else None
input_query = input('Enter recipe name query [optional]: ') if input('Do you want to enter a recipe name? (yes/no): ').lower() == 'yes' else None

# Find recommended recipes
recommended_recipes = recipe_finder.find_recipes(input_ingredients=input_ingredients, input_query=input_query, top_n=5)

# Print recommendations
print(recommended_recipes[['TranslatedRecipeName', 'TranslatedIngredients', 'URL']])


Do you want to enter ingredients? (yes/no):  no
Do you want to enter a recipe name? (yes/no):  nno


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

TypeError: unsupported operand type(s) for /: 'list' and 'int'